In [2]:
import pandas as pd
import os
from google.colab import drive
from collections import Counter
import re
import nltk # Mantenemos la importación de NLTK para la descarga, por si acaso

# --- DESCARGA Y CONFIGURACIÓN ---

# Forzar la descarga de stopwords (para reducir la chance de error)
try:
    nltk.download('stopwords', quiet=True)
except Exception:
    pass # Ignoramos el error si falla, porque usaremos la lista codificada

# Lista de stopwords en español (Solución robusta contra LookupError)
try:
    # Intenta cargar la lista de NLTK
    stop_words_es = set(nltk.corpus.stopwords.words('spanish'))
except LookupError:
    # Si falla, usa la lista codificada
    print("ADVERTENCIA: Usando lista de stopwords codificada para asegurar la ejecución.")
    stop_words_es = {'un', 'una', 'unas', 'unos', 'uno', 'sobre', 'también', 'tras', 'otro', 'otras', 'otros', 'otra', 'de', 'la', 'lo', 'las', 'los', 'a', 'ante', 'bajo', 'cabe', 'con', 'contra', 'desde', 'durante', 'en', 'entre', 'hacia', 'hasta', 'mediante', 'para', 'por', 'según', 'sin', 'so', 'sobre', 'tras', 'o', 'u', 'el', 'al', 'del', 'mi', 'mis', 'tu', 'tus', 'su', 'sus', 'aquel', 'aquella', 'aquellos', 'aquellas', 'este', 'esta', 'estos', 'estas', 'es', 'sea', 'pero', 'mas', 'más', 'y', 'e', 'si', 'sino', 'sólo', 'solo', 'tal', 'vez', 'toda', 'todo', 'todos', 'todas', 'qué', 'cual', 'cuales', 'quien', 'quienes', 'yo', 'me', 'mi', 'mío', 'mía', 'míos', 'mías', 'tú', 'te', 'ti', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'él', 'ella', 'ello', 'nos', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vos', 'vosotros', 'vosotras', 'vuestro', 'vuestra', 'vuestros', 'vuestras'}


# Montar Google Drive
try:
    drive.mount('/content/drive')
except:
    print("Drive ya montado.")

# Ruta a la carpeta que contiene tus archivos .txt
transcripciones_path = "/content/drive/MyDrive/TFG_2025/Entrevistas/"

# Diccionario de conceptos clave para el TFG
conceptos_clave = {
    'Estrés_Ansiedad': ['estrés', 'ansiedad', 'nervioso', 'agobio', 'presión', 'malo', 'difícil'],
    'Recuperación_Calma': ['calma', 'relajación', 'tranquilidad', 'paz', 'descanso', 'mejor'],
    'Metacognición_Reflexión': ['sentir', 'pensar', 'darse', 'cuenta', 'conciencia', 'reflejar', 'mente', 'proceso'],
    'Intervención_Sonido': ['sonido', 'música', 'paisaje', 'audio'],
}


def clean_and_tokenize(text):
    # Usamos REGEX para tokenizar (soluciona el LookupError de Punkt)
    tokens = re.findall(r'[a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]+', text.lower())
    # Limpieza: eliminar stopwords y palabras irrelevantes
    tokens = [word for word in tokens if word not in stop_words_es]
    tokens = [word for word in tokens if word not in ('gracias', 'sí', 'no', 'vale', 'entonces', 'pues', 'claro', 'eh', 'e', 'o')]
    return tokens

# 1. Crear el DataFrame de conteo por participante
df_conteo = []

for filename in os.listdir(transcripciones_path):
    if filename.endswith(".txt"):
        ruta_archivo = os.path.join(transcripciones_path, filename)

        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            content = f.read()

        tokens = clean_and_tokenize(content)
        word_counts = Counter(tokens)

        # Obtener el ID del participante (basado en el nombre del archivo)
        participante_data = {'ID_Participante': filename.split('.')[0]}

        # Contar la frecuencia de cada concepto en esta entrevista
        for concepto, palabras in conceptos_clave.items():
            count = sum(word_counts[palabra] for palabra in palabras)
            participante_data[concepto] = count

        df_conteo.append(participante_data)

df_qualitativo = pd.DataFrame(df_conteo)

# Guardar el Dataframe de conteo en Drive
output_path_qual = "/content/drive/MyDrive/TFG_2025/Conteo_Conceptos_Participante.csv"
df_qualitativo.to_csv(output_path_qual, index=False)

print("\n--- BASE DE DATOS CUALITATIVA CREADA ---")
print(f"DataFrame guardado en: {output_path_qual}")
print(df_qualitativo)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- BASE DE DATOS CUALITATIVA CREADA ---
DataFrame guardado en: /content/drive/MyDrive/TFG_2025/Conteo_Conceptos_Participante.csv
   ID_Participante  Estrés_Ansiedad  Recuperación_Calma  \
0    ale2311092025                2                   3   
1    rue1022092025                2                   2   
2    lli2108092025                5                   7   
3    ben2004092025                8                   7   
4    any0828082025                5                   4   
5    vas2804092025                9                   6   
6    yes1904092025                6                   8   
7    asi1621082025                2                   7   
8    bio0921082025                2                   6   
9    ESI0821082025                8                   6   
10   uro1628082025                5                   4   
11   osa1111092025              